In [ ]:
import sys
import cv2
import numpy as np
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QPushButton, QFileDialog, QComboBox, \
    QHBoxLayout, QGridLayout
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import Qt, QTimer
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load models here
model1 = load_model('Final_Facial_Expression_Detection_CNN.h5')  
model2 = load_model('Age_Model_res.h5')  

# Emotion labels
emotion_labels = ['Anger', 'Contempt', 'Disgust', 'Fear', 'Happy', 'Sadness', 'Surprise']

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


class EmotionEstimatorApp(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Face Expression Estimation")
        self.setGeometry(100, 100, 800, 600)  # Default window size

        # Layout setup
        self.layout = QGridLayout()
        self.setLayout(self.layout)

        # Dropdown to select the model
        self.model_dropdown = QComboBox()
        self.model_dropdown.addItem("CNN")
        self.model_dropdown.addItem("RES")
        self.layout.addWidget(self.model_dropdown, 0, 2)

        # Label for displaying selected model
        self.selected_model_label = QLabel("Model: CNN")
        self.layout.addWidget(self.selected_model_label, 0, 0)

        # Image Display
        self.image_label = QLabel("Image/Webcam")
        self.image_label.setAlignment(Qt.AlignCenter)
        self.layout.addWidget(self.image_label, 1, 0, 1, 3)

        # Buttons for actions
        self.upload_button = QPushButton("Upload Image")
        self.upload_button.clicked.connect(self.upload_image)
        self.layout.addWidget(self.upload_button, 8, 1)

        self.webcam_button = QPushButton("Open Webcam")
        self.webcam_button.clicked.connect(self.start_webcam)
        self.layout.addWidget(self.webcam_button, 6, 1)

        self.stop_webcam_button = QPushButton("Stop Webcam")
        self.stop_webcam_button.clicked.connect(self.stop_webcam)
        self.stop_webcam_button.setEnabled(False)  
        self.layout.addWidget(self.stop_webcam_button, 4, 1)

        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_webcam)

        self.capture = None

        self.model_dropdown.currentIndexChanged.connect(self.update_model)

    def update_model(self):
        """Handle model switching based on dropdown selection."""
        selected_model = self.model_dropdown.currentIndex()
        if selected_model == 0:
            self.selected_model_label.setText("Model: CNN")
        else:
            self.selected_model_label.setText("Model: RES")

    def upload_image(self):
        try:
            file_dialog = QFileDialog()
            image_path, _ = file_dialog.getOpenFileName()

            if image_path:
                self.process_image(image_path)
        except Exception as e:
            self.image_label.setText(f"Error uploading image: {e}")

    def start_webcam(self):
        try:
            self.capture = cv2.VideoCapture(0)
            if not self.capture.isOpened():
                self.image_label.setText("Failed to open webcam!")
                return
            self.webcam_button.setDisabled(True)  
            self.stop_webcam_button.setEnabled(True)
            self.timer.start(30)  
        except Exception as e:
            self.image_label.setText(f"Error starting webcam: {e}")

    def stop_webcam(self):
        """Stop the webcam and release resources."""
        try:
            if self.capture is not None:
                self.capture.release()
                self.capture = None
                self.timer.stop()  
                self.webcam_button.setEnabled(True)  
                self.stop_webcam_button.setEnabled(False)  
                self.image_label.setText("Image\Webcam")
            else:
                self.image_label.setText("No webcam feed to stop.")
        except Exception as e:
            self.image_label.setText(f"Error stopping webcam: {e}")

    def update_webcam(self):
        try:
            ret, frame = self.capture.read()
            if ret:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

                for (x, y, w, h) in faces:
                    face = frame_rgb[y:y + h, x:x + w]

                    predicted_emotion = self.predict_expression(face)

                    cv2.rectangle(frame_rgb, (x, y), (x + w, y + h), (0, 255, 0), 2)

                    cv2.putText(frame_rgb, predicted_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0),
                                2)

                self.display_image(frame_rgb)

            else:
                self.capture.release()
                self.capture = None
                self.webcam_button.setEnabled(True)
                self.stop_webcam_button.setEnabled(False)
                self.timer.stop()
        except Exception as e:
            self.image_label.setText(f"Error during webcam capture: {e}")

    def process_image(self, image_path):
        try:
            image = cv2.imread(image_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            for (x, y, w, h) in faces:
                face = image_rgb[y:y + h, x:x + w]

                predicted_emotion = self.predict_expression(face)

                cv2.rectangle(image_rgb, (x, y), (x + w, y + h), (0, 255, 0), 2)

                cv2.putText(image_rgb, predicted_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            self.display_image(image_rgb)

        except Exception as e:
            self.image_label.setText(f"Error processing image: {e}")

    def display_image(self, image):
        try:
            height, width, channel = image.shape
            bytes_per_line = 3 * width
            q_image = QImage(image.data, width, height, bytes_per_line, QImage.Format_RGB888)
            pixmap = QPixmap(q_image)
            self.image_label.setPixmap(
                pixmap.scaled(self.image_label.width(), self.image_label.height(), Qt.KeepAspectRatio))
        except Exception as e:
            self.image_label.setText(f"Error displaying image: {e}")

    def predict_expression(self, image):
        try:
            
            image_resized = cv2.resize(image, (48, 48))  
            image_array = img_to_array(image_resized)  
            image_array = np.expand_dims(image_array, axis=0)  
            image_array = image_array / 255.0 

            selected_model = self.model_dropdown.currentIndex()
            if selected_model == 0:
                model = model1
            else:
                model = model2

            # Make prediction
            prediction = model.predict(image_array)
            predicted_class = np.argmax(prediction, axis=1)[0]
            return emotion_labels[predicted_class]
        except Exception as e:
            self.image_label.setText(f"Error predicting emotion: {e}")
            return "Error"


if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = EmotionEstimatorApp()
    window.show()
    sys.exit(app.exec_())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


SystemExit: 0

c:\Users\nadee\anaconda3\envs\session2\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
